In [2]:
import pandas as pd
import time
import numpy as np
import threading

In [5]:
COL_MAX = get_column_count("../output/reduced_test_categorical.csv.gz")
start = time.time()
X = pd.read_csv("../output/reduced_test_categorical.csv.gz",
                compression="gzip",
                index_col=0,
                dtype=np.float32,
                usecols=[0, 247]
               )
print(time.time() - start)

3.4748950004577637


In [7]:
def get_column_count(filename):
    chunks = pd.read_csv(filename, chunksize=1)
    for c in chunks:
        return len(c.columns)


In [10]:
start = time.time()
Y = pd.read_csv("../output/reduced_test_categorical.csv.gz",
                compression="gzip",
                index_col=0,
                dtype=np.float32,
                usecols=np.concatenate([[0], list(range(226, 248))])
               )
print(time.time() - start)

4.898908853530884


In [27]:
filename = "../output/reduced_test_categorical.csv.gz"
COL_MAX = get_column_count(filename)
thread_num = 5
part_size = int(np.ceil(COL_MAX / thread_num))
begins = [0] * thread_num
ends = [0] * thread_num
for i in range(thread_num):
    if i == 0:
        begins[i] = 1
        ends[i] = part_size + 1
    else:
        begins[i] = ends[i-1]
        ends[i] = min(begins[i] + part_size + 1, COL_MAX)

readers = []
for i in range(thread_num):
    reader = ParallelCSVReader(filename, begins[i], ends[i])
    readers.append(reader)

for i in range(thread_num):
    readers[i].start()
    print(i)
    

0
1
2
3
4


In [24]:
begins, ends

([1, 51, 102, 153, 204], [51, 102, 153, 204, 248])

In [12]:
class ParallelCSVReader(threading.Thread):
    def __init__(self, filename, begin, end):
        super().__init__()
        self.filename = filename
        self.begin = begin
        self.end = end
        self.loaded = False
        self.df = None

    def run(self):
        cols = np.concatenate([[0], list(range(self.begin, self.end))])
        self.df = pd.read_csv(self.filename,
                              compression="gzip",
                              index_col=0,
                              dtype=np.float32,
                              usecols=cols
                              )
        self.loaded = True

    def get_df(self):
        return self.df


In [28]:
for i in range(thread_num):
    readers[i].join()
    print(i)
    

0
1
2
3
4


In [34]:
X = None
for i in range(thread_num):

    X = pd.concat([X, readers[i].get_df()], axis=1)
    

In [31]:
readers[0].get_df()

,L0_S1_F25,L0_S1_F27,L0_S1_F29,L0_S1_F31,L0_S2_F33,L0_S2_F35,L0_S4_F105,L0_S4_F107,L0_S4_F108,L0_S4_F110,...,L0_S14_F361,L0_S15_F396,L0_S16_F422,L0_S16_F424,L0_S16_F425,L0_S21_F468,L0_S21_F470,L0_S21_F471,L0_S21_F473,L0_S21_F475
Id,,,,,,,,,,,,,,,,,,,,,
1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
X

,L0_S1_F25,L0_S1_F27,L0_S1_F29,L0_S1_F31,L0_S2_F33,L0_S2_F35,L0_S4_F105,L0_S4_F107,L0_S4_F108,L0_S4_F110,...,L3_S43_F4064,L3_S43_F4081,L3_S43_F4084,L3_S44_F4102,L3_S46_F4134,L3_S47_F4141,L3_S49_F4207,L3_S49_F4210,L3_S49_F4217,L3_S49_F4220
Id,,,,,,,,,,,,,,,,,,,,,
1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN


In [1]:
from ParallelCSVReader import ParallelCSVReaderLoader

In [1]:
start = time.time()
loader = ParallelCSVReaderLoader(4, "../output/reduced_test_categorical.csv.gz")
loader.start()
loader.wait()
X = loader.concat_df()
print(time.time() - start)

NameError: name 'time' is not defined

In [8]:
start = time.time()
X = pd.read_csv("../output/reduced_test_categorical.csv.gz",
                compression="gzip",
                index_col=0,
                dtype=np.float32
               )
print(time.time() - start)

19.3104031085968
